# DSCI6003 Lab 4.3 - Implementing Bernoulli Bayes

# Please do this entire lab in two seperate .py files (bernoulli.py and run_bernoulli.py). The first will contain your implementation of the algorithm. The next, when I run it, should contain the scores of your code versus sklearn's. 

## Part 1: Implement the code.
Let's continue with the algorithm coding streak! We will be implementing Bernoulli Bayes today. There is a test file to make sure your code is implemented correctly.

In [1]:
import pandas as pd
import numpy as np

In [242]:
from bernoulli_JH import BernoulliBayes
from sklearn.model_selection import KFold
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from run_bernoulli import evaluate_code
from collections import Counter

In [3]:
%load_ext autoreload

In [212]:
%autoreload 2

In [135]:

from nose.tools import assert_equal
import nose

X = np.array([[1,0,1],
			  [1,0,0],
	 		  [0,1,0],
	 		  [1,1,0],
			  [0,1,1]])

y = np.array([0,0,1,1,1])

bb = BernoulliBayes()
bb.fit(X,y)

def test_priors():
    assert_equal(bb.prior[0],2)
    assert_equal(bb.prior[1],3)

def test_pvc():
    assert_equal(bb.pvc[0][0], 3./4)
    assert_equal(bb.pvc[0][1], 1./4)
    assert_equal(bb.pvc[0][2], 2./4)
    assert_equal(bb.pvc[1][0], 2./5)
    assert_equal(bb.pvc[1][1], 4./5)
    assert_equal(bb.pvc[1][2], 2./5)

def test_predict():
    assert_equal(sum(bb.predict(X)),3)



In [229]:
test_priors()

In [231]:
test_pvc()

In [232]:
test_predict()

## Part 2: Test it against sklearn.
We are going to use a dataset containing e-mails that are labeled as either spam(1) or not spam(0). Make sure your code is scoring similarly to sklearn using <a href = https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/>this dataset</a>. 

1. Load in the dataset using Pandas (please, I'm begging you use Pandas and make sure header = None).

2. Look at the data. We want to make sure we keep the columns that correspond to whether or not a word is present. Store the columns that contain information about whether or not a word is present. 

3. Create a threshold for your X values, so that anything above this threshold is a 1 and below is 0. You need to do this because BernoulliNB works on binarized data. 

4. Get the labels (last column). 

5. Run KFold cross validation with 5 folds and see how your score compares to sklearns.

6. Print out your accuracy against sklearns. 

7. Now, use sklearn's MultinomialNB classifier on this dataset. Does Bernoulli or Naive Bayes perform better? Try using different thresholds. Print out your conclusions (i.e. print("____ algorithm preforms better"))

In [263]:
spam_df = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data',header=None,delimiter=',')

In [264]:
#1 is spam 0 is not spam
spam_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
4596,0.31,0.0,0.62,0.0,0.00,0.31,0.0,0.0,0.0,0.0,...,0.000,0.232,0.0,0.000,0.0,0.0,1.142,3,88,0
4597,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0.353,0.0,0.0,1.555,4,14,0
4598,0.30,0.0,0.30,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.102,0.718,0.0,0.000,0.0,0.0,1.404,6,118,0
4599,0.96,0.0,0.00,0.0,0.32,0.00,0.0,0.0,0.0,0.0,...,0.000,0.057,0.0,0.000,0.0,0.0,1.147,5,78,0
4600,0.00,0.0,0.65,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0.125,0.0,0.0,1.250,5,40,0


In [265]:
spam_df.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [178]:
print([i for i in spam_df])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]


In [267]:
new_spam_df = np.zeros(np.shape(spam_df))
for col in spam_df:
    new_col = spam_df.iloc[:,col].apply(lambda x: 1 if x >0 else 0)
    new_spam_df[:,col]=new_col

In [272]:
X_spam = np.array(new_spam_df[:,:-1])


In [273]:
y_spam = np.array(new_spam_df[:,-1:]).ravel()


In [274]:
evaluate_code(X_spam,y_spam,5,.1)

My RMSE Score (Binomial NB) 0.34844770520737034
My accuracy score (Binomial NB):86.74%
SKlearn RMSE (Binomial NB):0.34844770520737034
SKlearn Accuracy (Binomial NB):86.74%
SKlearn Multi NB RMSE with 0.1 threshold:0.3465758149411503
SKlearn Multi NB Accuracy with 0.1 threshold:87.11%


In [275]:
evaluate_code(X_spam,y_spam,5,.5)

My RMSE Score (Binomial NB) 0.34844770520737034
My accuracy score (Binomial NB):86.74%
SKlearn RMSE (Binomial NB):0.34844770520737034
SKlearn Accuracy (Binomial NB):86.74%
SKlearn Multi NB RMSE with 0.5 threshold:0.3465758149411503
SKlearn Multi NB Accuracy with 0.5 threshold:87.11%


In [276]:
evaluate_code(X_spam,y_spam,5,2)

My RMSE Score (Binomial NB) 0.34844770520737034
My accuracy score (Binomial NB):86.74%
SKlearn RMSE (Binomial NB):0.34844770520737034
SKlearn Accuracy (Binomial NB):86.74%
SKlearn Multi NB RMSE with 2 threshold:0.3465758149411503
SKlearn Multi NB Accuracy with 2 threshold:87.11%


- Multi Naive bayes performs the best by a slight margin (~.4%). 